In [ ]:
library(tidyverse)
library(DescTools)
library(plotrix)
library(lsr)
library(pwr)
library(readxl)

For this notebook I'm going to use a few variables I pulled out of the American National Election Studies and cleaned and recoded.  The variables are:

Feeling thermometers (0-100 ratings on topics)
- "ft_sci" = Feelings about Scientists
- "ft_bigbusn" = Feelings about Big Business
- "ft_rich" = Feelings about Rich People
- "ft_congress" = Feelings about Congress

And two demographic variables (I've already collapsed into two groups)
- "race" = White vs. Not White
- "educ" = College Grad or higher vs. lower than college grad

In [ ]:
#load the .rds file
anes <- readRDS("anes.rds")
head(anes)

# Two Sample t-tests

## Independent Samples

We'll start by looking at two-sample t-tests for independent samples.  This is the most common type of two-sample t-test, used when there is no relationship between the two samples (they aren't paired in any way).

The easiest way to conduct a two-sample t-test in R when you have a dataframe of observations is to specify your test in "model format."  

This format is "dv ~ iv" where dv is the name of your dependent (numerical) variable and iv is the name of your independent variable (groups).


In [ ]:
## Conduct a t-test on the mean rating of scientists by education level
## general form - t.test(dv ~ iv, data)


t.test(ft_sci ~ educ, data = anes)

The results show us our typical output - tvalue, df, p-value.  This p-value indicates that there is a significant difference in the mean feeling about scientists between those who are lower educated and those with college degrees or higher.  The reported means for the two groups in the output show us that the higher rating is among college graduates or higher.

Remember that the alternative hypothesis here is just reporting the hypothesis tested (diff not equal to 0) and has no bearing on the result of the t-test.

The 95% confidence interval is the 95% confidence interval in the <b> difference </b> between the two means.  We know that the difference is signficant because the 95% confidence interval doesn't contain 0.

## Unequal Variances

You may have noticed above that R ran something called the "Welch Two Sample t-test," which is a form of the two-sample t-test that does not assume that the variances of the means of the two groups are equal.  

If we want to use the form of the calculation for equal variances we need to first test to see if our variances are equal.

This test looks at the ratio of our two variances: 

## $$\frac{\sigma_1^2}{\sigma_2^2}$$

The closer that this ratio is to 1, the more equal the variances are.

Because the test is looking at a ratio, we will use an F-value to test our hypothesis of equal variance - the F-value has two parameters, numerator degrees of freedom and denominator degrees of freedom.

The hypotheses for our test of equal variances are:

$H_0: var1 = var2 $ (equal variances)

$H_A: var1 \neq var2 $ (unequal variances)

In this case, failing to reject null is often seen as "good" because it means our variances are equal enough to use that version of the t-test.

If we reject null, we must use the version of the t-test that takes unequal variances into account.


In [ ]:
## test to see if our groups (educ) have equal variances for the ft rating about scientists
## general form - var.test(dv ~ iv)

var.test(anes$ft_sci ~ anes$educ)

The output shows us that the ratio of variances is 0.769 - meaning that the denominator variance was greater than the numerator variance.  

We get our F-value and our two degrees of freedom - one for the numerator and one for the denominator - these are n-1 for each of our groups we're comparing (college grad and greater vs. less than college grad)

Our p-value is very low, which means we reject the null hypothesis.  This provides support for the alternative hypothesis, which says that the true ratio of variances is not equal to 1 (unequal variances).  We can also see that our 95% confidence interval for the ratio of variances does not contain one, so they are significantly unequal.

This means that we do need to use the Welch Two-Sample t-test" which again, is the default in R `t.test()`.

If we wanted to run the t-test assuming equal variances, the code would be: 

In [ ]:
## Conduct a t-test on the mean rating of scientists by education level - assume equal variances
## general form - t.test(dv ~ iv, data, var.equal = TRUE)
t.test(ft_sci ~ educ, data = anes, var.equal = TRUE)

In this case our results are essentially similar, but there are slight differences in the values.

In [ ]:
# run both, save results
eqv <- t.test(ft_sci ~ educ, data = anes, var.equal = TRUE)
uneqv <- t.test(ft_sci ~ educ, data = anes)

In [ ]:
# Like before, our saved t-test has many saved pieces we can extract.  Let's look at the structure
str(eqv)

In [ ]:
# compare t-values
eqv$statistic
uneqv$statistic

In [ ]:
# compare degrees of freedom
eqv$parameter
uneqv$parameter
length(anes$ft_sci) ## n in dataframe

Why are our degrees of freedom different? The t-test that assumes equal variances in the two groups uses the expected n-2.  The Welch t-test has something lower, and it has decimal places?  This is because the degrees of freedom are approximated using a formula called the Welch–Satterthwaite equation.  All of the adjustments are made to account for the unequal variances.  

There is no benefit, however, to running the t-test with equal variances.  The Welch Two-Sample t-test works for both equal and unequal variances, which is why it is the default in R.

## Confidence Intervals
Like I said before the confidence intervals provided in the t-test output are the 95% CI for the <b><i> difference </b></i> in the means.  If this CI doesn't contain 0, the difference in the means is significantly different from zero.

Let's look at another variable to test, feeling rating about congress by race (white vs. non-white)

In [ ]:
## Conduct a t-test on the mean rating of congress by race
## general form - t.test(dv ~ iv, data)

t.test(ft_congress ~ race, data = anes) 
## don't need any options about equality of variances - we'll use the default that assumes they are not equal

The 95% CI for the difference in mean feeling about congress between whites and non-whites is from 4.73 to 8.18 points (on a scale from 0 to 100).

## Paired Samples t-tests
When our samples are not independent, they arepaired in some way, we need to use a special form of the t-test to account for the paired scores.  In most cases they are two measurements of the same variable taken from the same unit (same person) at two different times.  In other cases, it may be appropriate to use paired samples tests even if the measurements are not from the exact same person - such as if they are from spouses.  Or, in some cases medical trials pair together two individuals who match on many demographic characteristics and give one the treatement and one the control - since they were paired in that way and it determined their treatment they can't be considered independent.

For a paired t-test the $SE_{diff}$ is calculated with the difference between the pre- and post- scores, where $d$ is the difference, $n$ is the number of <b> pairs </b> and $df = n-1$ (number of pairs minus 1).  

# $$  t = \frac{\bar{X}_1 - \bar{X}_2}{\sqrt{\frac{n(\sum d^2) - (\sum d)^2}{n-1}}}  $$

This calculation should look familiar to the "computational" formula for variance we saw in the descriptive statistics lecture.  If we have pre- and post- scores from every person in the dataframe, then n-1 is number of people minus 1.

For this example I'm going to load in another cleaned set of variables from ANES, this time pre- and post-election feeling thermometer ratings of the democratic and republican presidential candidates from 2016 (Clinton and Trump).



In [ ]:
#load the .rds file
anes2 <- readRDS("anes2.rds")
head(anes2)

I'm going to first compare the mean rating of the democratic candidate, Clinton, pre- and post-election among all respondents.

In [ ]:
## paired t-test code -  t.test(pre, post, pooled=F, paired=T)
## pooled is F (false) because it's a single sample, repeated
## paired is T (true) because we're conducting a paired t-test

t.test(anes2$ft_pre_dem, anes2$ft_post_dem, pooled=F, paired=T)


This result again looks very similar to what we've seen previously, and our p-value is less than alpha = 0.05 so we can reject the hypothesis that the difference in means is 0, therefore we conclude that there is a statistically significant difference in the pre- and post-election ratings of Hilary Clinton.

Again, the confidence interval is for the difference in means - the negative number is a bit confusing - it comes from pre minus post, so the mean was higher in the post-election interview.  The mean displayed in the output is also just the difference, not the means of the two ratings.  If we're interested in those we can compare:

In [ ]:
# get means for pre- and post- Clinton ratings
mean(anes2$ft_pre_dem)
mean(anes2$ft_post_dem)
mean(anes2$ft_pre_dem) - mean(anes2$ft_post_dem) ## difference in means

We could also look at the difference within the party ids, but by running t-tests separately for each subgroup.

In [ ]:
## paired t-test filtered group of only democrats
dems <- anes2  %>% filter(partyid == "dem")
t.test(dems$ft_pre_dem, dems$ft_post_dem, pooled=F, paired=T)

For the democrats the difference in pre- and post-election ratings was much smaller, and is not significant at alpha = 0.05.  The p-value is greater than 0.05 and the confidence interval crosses over 0.  This means that there is NOT a statistically signficant difference between the pre- and post-election ratings of Clinton among democrats.

In [ ]:
## paired t-test filtered group of only republicans - ratings of Trump
reps <- anes2  %>% filter(partyid == "rep")
t.test(reps$ft_pre_rep, reps$ft_post_rep, pooled=F, paired=T)

In [ ]:
mean(reps$ft_pre_rep)
mean(reps$ft_post_rep)